# 🇰🇷 한국어 감성분석 (Korean Sentiment Analysis)

Google Colab에서 실행 가능한 한국어 텍스트 감성분석 시스템입니다.

## 📋 기능
- Excel 파일에서 한국어 텍스트 읽기
- 한국어 전용 감성분석 모델 사용
- 결과를 Excel 파일로 저장

## 🚀 사용법
1. Excel 파일을 업로드 (텍스트 컬럼 포함)
2. 코드 실행
3. 결과 다운로드

## 📦 필요한 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install torch transformers pandas openpyxl matplotlib seaborn wordcloud

## 🔧 라이브러리 임포트

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = ['DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 임포트 완료")

## 🤖 한국어 감성분석 모델 로드

In [ ]:
# 한국어 감성분석 모델 로드
model_name = "snunlp/KR-FinBert-SC"

print(f"📥 모델 다운로드 중: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✅ 모델 로드 완료 (장치: {device}")
print(f"📊 모델: {model_name}")

## 📁 Excel 파일 업로드

In [ ]:
from google.colab import files

print("📁 Excel 파일을 업로드해주세요...")
uploaded = files.upload()

# 업로드된 파일명 확인
file_name = list(uploaded.keys())[0]
print(f"✅ 파일 업로드 완료: {file_name}")

## 📊 Excel 파일 읽기

In [ ]:
# Excel 파일 읽기
df = pd.read_excel(file_name)
print(f"📊 데이터 로드 완료: {len(df)} 행")
print(f"📋 컬럼: {list(df.columns)}")
print("\n🔍 데이터 미리보기:")
display(df.head())

## 🎯 텍스트 컬럼 선택

In [ ]:
# 텍스트 컬럼 선택 (자동 감지)
text_columns = []
for col in df.columns:
    if df[col].dtype == 'object':  # 문자열 타입 컬럼
        text_columns.append(col)

if text_columns:
    text_column = text_columns[0]  # 첫 번째 텍스트 컬럼 사용
    print(f"✅ 텍스트 컬럼 선택: {text_column}")
else:
    print("❌ 텍스트 컬럼을 찾을 수 없습니다.")
    text_column = None

if text_column:
    print(f"\n📝 텍스트 샘플:")
    for i, text in enumerate(df[text_column].head()):
        print(f"{i+1}. {text}")

## 🧠 감성분석 실행

In [ ]:
def analyze_sentiment(text):
    """텍스트의 감성을 분석합니다."""
    if pd.isna(text) or str(text).strip() == "":
        return "neutral", 0.0
    
    # 토큰화
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1)
        
    # 결과 해석
    scores = probabilities[0].cpu().numpy()
    
    # KR-FinBert-SC: [negative, positive, neutral]
    labels = ["negative", "positive", "neutral"]
    predicted_label = labels[np.argmax(scores)]
    confidence = float(np.max(scores))
    
    return predicted_label, confidence

# 감성분석 실행
print("🧠 감성분석 실행 중...")
results = []

for idx, text in enumerate(df[text_column]):
    sentiment, confidence = analyze_sentiment(text)
    results.append({
        'text': text,
        'sentiment': sentiment,
        'confidence': confidence
    })
    
    if (idx + 1) % 10 == 0:
        print(f"진행률: {idx + 1}/{len(df)} ({((idx + 1)/len(df)*100):.1f}%)")

print("✅ 감성분석 완료!")

## 📊 결과 표시

In [ ]:
# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)
results_df['original_text'] = df[text_column]

# 감성별 통계
sentiment_counts = results_df['sentiment'].value_counts()
print("📊 감성분석 결과:")
print(f"총 텍스트 수: {len(results_df)}")
print("\n감성별 분포:")
for sentiment, count in sentiment_counts.items():
    percentage = (count / len(results_df)) * 100
    print(f"- {sentiment}: {count}개 ({percentage:.1f}%)")

print("\n🔍 상세 결과:")
display(results_df.head(10))

## 📈 시각화

In [ ]:
# 1. 감성별 막대 차트
plt.figure(figsize=(10, 6))
sentiment_counts.plot(kind='bar', color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
plt.title('한국어 텍스트 감성분석 결과', fontsize=16, fontweight='bold')
plt.xlabel('감성')
plt.ylabel('텍스트 수')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. 감성별 파이 차트
plt.figure(figsize=(8, 8))
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']
plt.pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%', colors=colors)
plt.title('감성별 비율', fontsize=16, fontweight='bold')
plt.axis('equal')
plt.show()

# 3. 신뢰도 분포
plt.figure(figsize=(10, 6))
plt.hist(results_df['confidence'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('감성분석 신뢰도 분포', fontsize=16, fontweight='bold')
plt.xlabel('신뢰도')
plt.ylabel('빈도')
plt.tight_layout()
plt.show()

print("✅ 차트 생성 완료!")

## 💾 결과 저장

In [ ]:
# 결과를 Excel 파일로 저장
output_filename = f"korean_sentiment_analysis_result_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    # 원본 데이터와 결과
    combined_df = df.copy()
    combined_df['감성분석_결과'] = results_df['sentiment']
    combined_df['신뢰도'] = results_df['confidence']
    combined_df.to_excel(writer, sheet_name='감성분석_결과', index=False)
    
    # 감성별 통계
    stats_df = pd.DataFrame({
        '감성': sentiment_counts.index,
        '텍스트_수': sentiment_counts.values,
        '비율(%)': [(count / len(results_df)) * 100 for count in sentiment_counts.values]
    })
    stats_df.to_excel(writer, sheet_name='감성별_통계', index=False)

print(f"✅ 결과 저장 완료: {output_filename}")
print("📁 파일을 다운로드하려면 다음 셀을 실행하세요.")

## 📥 결과 파일 다운로드

In [ ]:
# 결과 파일 다운로드
files.download(output_filename)
print("✅ 파일 다운로드 완료!")

## 🎉 완료!

한국어 텍스트 감성분석이 성공적으로 완료되었습니다!

### 📋 수행된 작업:
1. ✅ 한국어 감성분석 모델 로드
2. ✅ Excel 파일 업로드 및 읽기
3. ✅ 텍스트 컬럼 자동 감지
4. ✅ 감성분석 실행
5. ✅ 결과 시각화
6. ✅ Excel 파일로 결과 저장
7. ✅ 결과 파일 다운로드

### 💡 다음에 사용할 때:
- 이 노트북을 복사하여 새로운 분석에 사용
- 다른 한국어 텍스트 데이터로 분석 가능
- 모델은 한 번만 다운로드하면 됨